# Hybrid Multi-Robot Coordination System for Package Delivery

This notebook implements a Hybrid Multi-Robot Coordination System for the package delivery problem, combining centralized task allocation with decentralized execution. This approach does not require any training, instead relying on well-established optimization techniques.

## Key Components:

1. **Centralized Task Manager (CTM)**
   - Package prioritization based on urgency and value
   - Cost matrix construction for optimal assignment
   - Hungarian algorithm for optimal robot-package assignment

2. **Decentralized Robot Execution**
   - A* pathfinding for optimal routes
   - Collision avoidance with prioritized planning
   - Dynamic replanning when paths are blocked

3. **Global Optimization**
   - Congestion management
   - Efficient data structures
   - Special case handling

In [1]:
# Import necessary libraries
import numpy as np
import heapq
import time
import matplotlib.pyplot as plt
from collections import defaultdict, deque
import copy

# Import the environment from the existing code
from env import Environment

## 1. Centralized Task Manager (CTM)

The Centralized Task Manager (CTM) is responsible for assigning packages to robots. It uses a number of optimization techniques to ensure efficient package delivery.

In [2]:
class CentralizedTaskManager:
    """
    Centralized Task Manager for optimal package assignment
    """
    def __init__(self, env, reassign_interval=5, w_time=0.01, w_reward=1.0, w_penalty=0.1):
        """
        Initialize the Centralized Task Manager
        
        Parameters:
        -----------
        env : Environment
            The environment instance
        reassign_interval : int
            How often to reassign tasks (in timesteps)
        w_time : float
            Weight for time cost in assignment calculation
        w_reward : float
            Weight for reward in assignment calculation
        w_penalty : float
            Weight for deadline penalty in assignment calculation
        """
        self.env = env
        self.map = np.array(env.grid)
        self.n_rows, self.n_cols = self.map.shape
        self.n_robots = len(env.robots)
        self.reassign_interval = reassign_interval
        self.w_time = w_time
        self.w_reward = w_reward
        self.w_penalty = w_penalty
        
        # Robot assignments: robot_id -> package_id or None if not assigned
        self.robot_assignments = {i: None for i in range(self.n_robots)}
        
        # Package assignments: package_id -> robot_id or None if not assigned
        self.package_assignments = {}
        
        # Precompute distance matrix for pathfinding
        self.distance_matrix = self.precompute_distances()
        
        # Store paths for each robot
        self.robot_paths = {i: [] for i in range(self.n_robots)}
        
        # Last assignment time
        self.last_assignment_time = -1
        
    def precompute_distances(self):
        """
        Precompute distances between all valid cells using BFS
        
        Returns:
        --------
        dict : Dictionary mapping (start_pos) -> {end_pos: distance}
        """
        distance_matrix = {}
        valid_cells = [(i, j) for i in range(self.n_rows) for j in range(self.n_cols) 
                      if self.map[i, j] == 0]
        
        for start in valid_cells:
            distances = {}
            queue = deque([(start, 0)])
            visited = {start}
            
            while queue:
                pos, dist = queue.popleft()
                distances[pos] = dist
                
                for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
                    new_pos = (pos[0] + dx, pos[1] + dy)
                    if (0 <= new_pos[0] < self.n_rows and 0 <= new_pos[1] < self.n_cols and 
                        self.map[new_pos] == 0 and new_pos not in visited):
                        visited.add(new_pos)
                        queue.append((new_pos, dist + 1))
            
            distance_matrix[start] = distances
            
        return distance_matrix
    
    def get_distance(self, pos1, pos2):
        """
        Get the distance between two positions using the precomputed distance matrix
        
        Parameters:
        -----------
        pos1 : tuple
            Starting position (row, col)
        pos2 : tuple
            Target position (row, col)
            
        Returns:
        --------
        int : Distance between positions, or float('inf') if no path exists
        """
        if pos1 in self.distance_matrix and pos2 in self.distance_matrix[pos1]:
            return self.distance_matrix[pos1][pos2]
        return float('inf')  # No path exists
    
    def prioritize_packages(self, current_time):
        """
        Prioritize packages based on urgency and value
        
        Parameters:
        -----------
        current_time : int
            Current timestep
            
        Returns:
        --------
        list : List of package IDs sorted by priority (highest first)
        """
        waiting_packages = []
        
        for pkg in self.env.packages:
            # Only consider packages that have appeared and are not yet delivered
            if pkg.start_time <= current_time and pkg.status != 'delivered':
                if pkg.status == 'in_transit':
                    # If package is already being carried, skip it
                    continue
                
                # Calculate urgency score based on remaining time to deadline
                time_remaining = max(1, pkg.deadline - current_time)
                urgency_score = 1.0 / time_remaining
                
                # Calculate value score
                if current_time + self.get_distance(pkg.start, pkg.target) <= pkg.deadline:
                    value_score = self.env.delivery_reward  # On-time delivery reward
                else:
                    value_score = self.env.delay_reward  # Late delivery reward
                
                # Calculate priority score (higher is better)
                priority_score = 0.7 * urgency_score + 0.3 * value_score
                
                waiting_packages.append((pkg.package_id, priority_score))
        
        # Sort by priority score (highest first)
        waiting_packages.sort(key=lambda x: x[1], reverse=True)
        
        return [pkg_id for pkg_id, _ in waiting_packages]
    
    def build_cost_matrix(self, current_time):
        """
        Build cost matrix for assignment optimization
        
        Parameters:
        -----------
        current_time : int
            Current timestep
            
        Returns:
        --------
        numpy.ndarray : Cost matrix for Hungarian algorithm
        list : List of eligible robot IDs
        list : List of eligible package IDs
        """
        # Get prioritized packages
        prioritized_packages = self.prioritize_packages(current_time)
        
        # Determine available robots
        available_robots = []
        for robot_id in range(self.n_robots):
            robot = self.env.robots[robot_id]
            if robot.carrying == 0:  # Robot is not carrying a package
                available_robots.append(robot_id)
            else:
                # If robot is carrying a package, keep its current assignment
                pass
        
        # If no robots or packages are available, return empty matrices
        if not available_robots or not prioritized_packages:
            return np.array([]), [], []
        
        # Create cost matrix
        n_robots = len(available_robots)
        n_packages = len(prioritized_packages)
        cost_matrix = np.full((n_robots, n_packages), float('inf'))
        
        for i, robot_id in enumerate(available_robots):
            robot = self.env.robots[robot_id]
            robot_pos = robot.position
            
            for j, pkg_id in enumerate(prioritized_packages):
                pkg = self.env.packages[pkg_id - 1]  # Adjust for 1-indexing
                
                # Skip packages that are already in transit
                if pkg.status == 'in_transit':
                    continue
                
                # Calculate time costs
                time_to_pickup = self.get_distance(robot_pos, pkg.start)
                time_pickup_to_dropoff = self.get_distance(pkg.start, pkg.target)
                
                # If either path is impossible, set cost to infinity
                if time_to_pickup == float('inf') or time_pickup_to_dropoff == float('inf'):
                    cost_matrix[i, j] = float('inf')
                    continue
                
                total_travel_steps = time_to_pickup + time_pickup_to_dropoff
                
                # Calculate reward and deadline penalty
                estimated_completion_time = current_time + total_travel_steps
                if estimated_completion_time <= pkg.deadline:
                    effective_reward = self.env.delivery_reward
                    deadline_penalty = 0
                else:
                    effective_reward = self.env.delay_reward
                    deadline_penalty = estimated_completion_time - pkg.deadline
                
                # Calculate total cost (lower is better for Hungarian algorithm)
                cost = (self.w_time * total_travel_steps) - \
                       (self.w_reward * effective_reward) + \
                       (self.w_penalty * deadline_penalty)
                
                cost_matrix[i, j] = cost
        
        return cost_matrix, available_robots, prioritized_packages
    
    def hungarian_algorithm(self, cost_matrix):
        """
        Implementation of the Hungarian algorithm for optimal assignment
        
        Parameters:
        -----------
        cost_matrix : numpy.ndarray
            Cost matrix where rows represent robots and columns represent packages
            
        Returns:
        --------
        list : List of (row, col) tuples representing assignments
        """
        # If matrix is empty, return empty assignment
        if cost_matrix.size == 0:
            return []
        
        # Handle non-square matrices by padding with zeros
        n_rows, n_cols = cost_matrix.shape
        if n_rows > n_cols:
            # More workers than jobs - pad with dummy columns
            padding = np.zeros((n_rows, n_rows - n_cols))
            cost_matrix = np.hstack((cost_matrix, padding))
        elif n_cols > n_rows:
            # More jobs than workers - pad with dummy rows
            padding = np.zeros((n_cols - n_rows, n_cols))
            cost_matrix = np.vstack((cost_matrix, padding))
        
        # Make a copy of the cost matrix
        cost_matrix = cost_matrix.copy()
        
        # Step 1: Subtract row minima
        for i in range(cost_matrix.shape[0]):
            cost_matrix[i, :] -= np.min(cost_matrix[i, :])
        
        # Step 2: Subtract column minima
        for j in range(cost_matrix.shape[1]):
            cost_matrix[:, j] -= np.min(cost_matrix[:, j])
        
        # Step 3-5: Cover all zeros with minimum number of lines and adjust matrix
        n = cost_matrix.shape[0]
        assigned = np.zeros(cost_matrix.shape, dtype=int)
        row_assignments = [-1] * n
        col_assignments = [-1] * n
        
        # Initial assignment
        for i in range(n):
            for j in range(n):
                if cost_matrix[i, j] == 0 and row_assignments[i] == -1 and col_assignments[j] == -1:
                    assigned[i, j] = 1
                    row_assignments[i] = j
                    col_assignments[j] = i
        
        while -1 in row_assignments:
            # Find uncovered zeros and augment the assignments
            zero_paths = self.find_augmenting_paths(cost_matrix, assigned)
            
            if not zero_paths:
                # No augmenting paths found, need to adjust the cost matrix
                # Find min value in uncovered cells
                row_cover = [1 if assignment != -1 else 0 for assignment in row_assignments]
                col_cover = [1 if assignment != -1 else 0 for assignment in col_assignments]
                
                uncovered_indices = [(i, j) for i in range(n) for j in range(n)
                                    if row_cover[i] == 0 and col_cover[j] == 0]
                
                if not uncovered_indices:
                    break
                
                min_val = min(cost_matrix[i, j] for i, j in uncovered_indices)
                
                # Subtract min_val from uncovered rows
                for i in range(n):
                    if row_cover[i] == 0:
                        cost_matrix[i, :] -= min_val
                
                # Add min_val to covered columns
                for j in range(n):
                    if col_cover[j] == 1:
                        cost_matrix[:, j] += min_val
                
            else:
                # Augment assignments along the path
                for path in zero_paths:
                    for i, j in path:
                        assigned[i, j] = 1 - assigned[i, j]  # Flip assignment
                    
                    # Update row_assignments and col_assignments
                    for i in range(n):
                        row_assignments[i] = -1
                        
                    for j in range(n):
                        col_assignments[j] = -1
                    
                    for i in range(n):
                        for j in range(n):
                            if assigned[i, j] == 1:
                                row_assignments[i] = j
                                col_assignments[j] = i
        
        # Extract and return the assignments
        assignments = []
        for i in range(min(n_rows, n)):
            if row_assignments[i] != -1 and row_assignments[i] < n_cols:
                assignments.append((i, row_assignments[i]))
        
        return assignments
    
    def find_augmenting_paths(self, cost_matrix, assigned):
        """
        Find augmenting paths for the Hungarian algorithm
        
        Parameters:
        -----------
        cost_matrix : numpy.ndarray
            Cost matrix
        assigned : numpy.ndarray
            Assignment matrix
            
        Returns:
        --------
        list : List of augmenting paths
        """
        n = cost_matrix.shape[0]
        row_assignments = [-1] * n
        col_assignments = [-1] * n
        
        # Update row_assignments and col_assignments based on assigned
        for i in range(n):
            for j in range(n):
                if assigned[i, j] == 1:
                    row_assignments[i] = j
                    col_assignments[j] = i
        
        # Find unassigned rows
        unassigned_rows = [i for i in range(n) if row_assignments[i] == -1]
        
        paths = []
        for row in unassigned_rows:
            # Initialize path
            path = []
            visited_rows = set()
            visited_cols = set()
            
            # Start DFS from the unassigned row
            if self.find_path(row, cost_matrix, assigned, row_assignments, col_assignments,
                           visited_rows, visited_cols, path):
                paths.append(path)
        
        return paths
    
    def find_path(self, row, cost_matrix, assigned, row_assignments, col_assignments,
                 visited_rows, visited_cols, path):
        """
        Helper function for find_augmenting_paths
        
        Parameters:
        -----------
        row : int
            Current row
        cost_matrix : numpy.ndarray
            Cost matrix
        assigned : numpy.ndarray
            Assignment matrix
        row_assignments : list
            Row assignments
        col_assignments : list
            Column assignments
        visited_rows : set
            Set of visited rows
        visited_cols : set
            Set of visited columns
        path : list
            Current path
            
        Returns:
        --------
        bool : True if a path was found, False otherwise
        """
        if row in visited_rows:
            return False
        
        visited_rows.add(row)
        
        for col in range(cost_matrix.shape[1]):
            if cost_matrix[row, col] == 0 and col not in visited_cols:
                path.append((row, col))
                
                # If this is an unassigned column, we've found a path
                if col_assignments[col] == -1:
                    return True
                
                # Otherwise, continue the search from the assigned row
                if self.find_path(col_assignments[col], cost_matrix, assigned,
                               row_assignments, col_assignments,
                               visited_rows, visited_cols, path):
                    return True
                
                # If no path found, backtrack
                path.pop()
        
        return False
    
    def assign_tasks(self, current_time, force_reassign=False):
        """
        Assign tasks to robots using the Hungarian algorithm
        
        Parameters:
        -----------
        current_time : int
            Current timestep
        force_reassign : bool
            Whether to force reassignment regardless of interval
            
        Returns:
        --------
        dict : Updated robot assignments
        """
        # Check if it's time to reassign
        if not force_reassign and current_time - self.last_assignment_time < self.reassign_interval:
            return self.robot_assignments
        
        self.last_assignment_time = current_time
        
        # Build cost matrix
        cost_matrix, available_robots, prioritized_packages = self.build_cost_matrix(current_time)
        
        # If no assignments possible, return current assignments
        if cost_matrix.size == 0:
            return self.robot_assignments
        
        # Run Hungarian algorithm
        assignments = self.hungarian_algorithm(cost_matrix)
        
        # Update assignments
        for i, j in assignments:
            robot_id = available_robots[i]
            pkg_id = prioritized_packages[j]
            
            # Cost infinity means this assignment is impossible
            if cost_matrix[i, j] == float('inf'):
                continue
            
            pkg = self.env.packages[pkg_id - 1]  # Adjust for 1-indexing
            
            # Only assign if package is waiting
            if pkg.status == 'waiting':
                self.robot_assignments[robot_id] = pkg_id
                self.package_assignments[pkg_id] = robot_id
        
        return self.robot_assignments
    
    def get_robot_destination(self, robot_id):
        """
        Get the destination for a robot based on its current assignment
        
        Parameters:
        -----------
        robot_id : int
            Robot ID
            
        Returns:
        --------
        tuple : Destination (row, col) or None if no destination
        str : Task type ('pickup' or 'dropoff')
        """
        package_id = self.robot_assignments[robot_id]
        
        if package_id is None:
            return None, None
        
        robot = self.env.robots[robot_id]
        package = self.env.packages[package_id - 1]  # Adjust for 1-indexing
        
        # If robot is not carrying a package, destination is pickup location
        if robot.carrying == 0:
            return package.start, 'pickup'
        
        # If robot is carrying a package, destination is dropoff location
        if robot.carrying == package_id:
            return package.target, 'dropoff'
        
        # If robot is carrying a different package, clear this assignment
        self.robot_assignments[robot_id] = None
        return None, None
    
    def update_assignments(self, current_time):
        """
        Update assignments based on current environment state
        
        Parameters:
        -----------
        current_time : int
            Current timestep
            
        Returns:
        --------
        dict : Updated robot assignments
        """
        # Update assignments for robots that have completed their tasks
        for robot_id in range(self.n_robots):
            package_id = self.robot_assignments[robot_id]
            
            if package_id is not None:
                package = self.env.packages[package_id - 1]  # Adjust for 1-indexing
                
                # If package is delivered, clear assignment
                if package.status == 'delivered':
                    self.robot_assignments[robot_id] = None
                    
                    if package_id in self.package_assignments:
                        del self.package_assignments[package_id]
        
        # Periodically reassign tasks
        if current_time % self.reassign_interval == 0:
            return self.assign_tasks(current_time, force_reassign=True)
        
        # Handle robots that have become available
        for robot_id in range(self.n_robots):
            if self.robot_assignments[robot_id] is None:
                # This robot needs a new assignment
                pass
        
        return self.robot_assignments

## 2. Decentralized Robot Execution

Each robot executes its assigned tasks independently, using A* pathfinding to navigate to pickup and dropoff locations.

In [3]:
class RobotController:
    """
    Decentralized Robot Controller for path planning and execution
    """
    def __init__(self, robot_id, env, ctm):
        """
        Initialize a Robot Controller
        
        Parameters:
        -----------
        robot_id : int
            Robot ID
        env : Environment
            Environment instance
        ctm : CentralizedTaskManager
            Centralized Task Manager instance
        """
        self.robot_id = robot_id
        self.env = env
        self.ctm = ctm
        self.map = np.array(env.grid)
        self.n_rows, self.n_cols = self.map.shape
        
        # Current path
        self.current_path = []
        self.destination = None
        self.task_type = None
        
        # Space-time reservations for collision avoidance
        self.reservations = {}  # (position, time) -> robot_id
    
    def a_star_search(self, start, goal, current_time, other_reservations):
        """
        A* search algorithm for path planning with space-time reservations
        
        Parameters:
        -----------
        start : tuple
            Starting position (row, col)
        goal : tuple
            Goal position (row, col)
        current_time : int
            Current timestep
        other_reservations : dict
            Space-time reservations from other robots
            
        Returns:
        --------
        list : List of positions forming the path
        """
        if start == goal:
            return [start]
        
        # Initialize data structures
        open_set = []
        closed_set = set()
        came_from = {}
        g_score = {start: 0}
        f_score = {start: self.heuristic(start, goal)}
        
        # Initial space-time position (position, time)
        heapq.heappush(open_set, (f_score[start], start, current_time))
        
        while open_set:
            # Get the node with the lowest f_score
            _, current, time = heapq.heappop(open_set)
            
            # Check if we've reached the goal
            if current == goal:
                # Reconstruct the path
                path = self.reconstruct_path(came_from, current)
                return path
            
            # Mark as visited
            closed_set.add((current, time))
            
            # Try each neighbor
            for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1), (0, 0)]:  # Up, Down, Left, Right, Stay
                neighbor = (current[0] + dx, current[1] + dy)
                next_time = time + 1
                
                # Check if the neighbor is valid
                if not self.is_valid_position(neighbor):
                    continue
                
                # Check for space-time collision
                if (neighbor, next_time) in other_reservations:
                    if other_reservations[(neighbor, next_time)] < self.robot_id:
                        # Higher priority robot has reserved this space-time
                        continue
                
                # Check if this space-time has been visited
                if (neighbor, next_time) in closed_set:
                    continue
                
                # Calculate tentative g_score
                tentative_g_score = g_score[current] + 1  # Uniform cost
                
                # If neighbor not in g_score or we found a better path
                if neighbor not in g_score or tentative_g_score < g_score[neighbor]:
                    # Update path
                    came_from[neighbor] = current
                    g_score[neighbor] = tentative_g_score
                    f_score[neighbor] = tentative_g_score + self.heuristic(neighbor, goal)
                    
                    # Add to open set if not there
                    if all((f, n, t) != (f_score[neighbor], neighbor, next_time) for f, n, t in open_set):
                        heapq.heappush(open_set, (f_score[neighbor], neighbor, next_time))
        
        # No path found
        return []
    
    def heuristic(self, a, b):
        """
        Manhattan distance heuristic for A*
        
        Parameters:
        -----------
        a : tuple
            First position (row, col)
        b : tuple
            Second position (row, col)
            
        Returns:
        --------
        int : Manhattan distance between positions
        """
        return abs(a[0] - b[0]) + abs(a[1] - b[1])
    
    def reconstruct_path(self, came_from, current):
        """
        Reconstruct the path from A* search
        
        Parameters:
        -----------
        came_from : dict
            Dictionary mapping positions to their predecessor
        current : tuple
            Current position
            
        Returns:
        --------
        list : List of positions forming the path
        """
        total_path = [current]
        while current in came_from:
            current = came_from[current]
            total_path.append(current)
        
        return total_path[::-1]  # Reverse to get path from start to goal
    
    def is_valid_position(self, position):
        """
        Check if a position is valid (within bounds and not an obstacle)
        
        Parameters:
        -----------
        position : tuple
            Position to check (row, col)
            
        Returns:
        --------
        bool : True if position is valid, False otherwise
        """
        row, col = position
        # Check if within bounds
        if row < 0 or row >= self.n_rows or col < 0 or col >= self.n_cols:
            return False
        
        # Check if obstacle
        if self.map[row, col] == 1:
            return False
        
        return True
    
    def update_path(self, current_time, other_reservations):
        """
        Update the robot's path based on current assignment
        
        Parameters:
        -----------
        current_time : int
            Current timestep
        other_reservations : dict
            Space-time reservations from other robots
            
        Returns:
        --------
        bool : True if path was updated, False otherwise
        """
        # Get current position and assignment
        robot = self.env.robots[self.robot_id]
        current_pos = robot.position
        destination, task_type = self.ctm.get_robot_destination(self.robot_id)
        
        # If no destination, clear path
        if destination is None:
            self.current_path = []
            self.destination = None
            self.task_type = None
            return False
        
        # If destination changed, recompute path
        if destination != self.destination or task_type != self.task_type:
            self.destination = destination
            self.task_type = task_type
            self.current_path = self.a_star_search(current_pos, destination, current_time, other_reservations)
            return True
        
        # If we have a path and it's still valid, keep it
        if self.current_path and self.current_path[0] == current_pos:
            # Check if path is still valid
            for i, pos in enumerate(self.current_path[1:], 1):
                if (pos, current_time + i) in other_reservations:
                    if other_reservations[(pos, current_time + i)] < self.robot_id:
                        # Higher priority robot has reserved this space-time, recompute path
                        self.current_path = self.a_star_search(current_pos, destination, current_time, other_reservations)
                        return True
            
            # Path is still valid
            return False
        
        # Recompute path
        self.current_path = self.a_star_search(current_pos, destination, current_time, other_reservations)
        return True
    
    def get_next_action(self, current_time):
        """
        Get the next action for the robot
        
        Parameters:
        -----------
        current_time : int
            Current timestep
            
        Returns:
        --------
        tuple : (move_action, package_action)
        list : List of space-time reservations
        """
        # Get current position and assignment
        robot = self.env.robots[self.robot_id]
        current_pos = robot.position
        
        # If no path, stay put
        if not self.current_path or len(self.current_path) <= 1:
            return ('S', '0'), []
        
        # Get next position in path
        next_pos = self.current_path[1] if len(self.current_path) > 1 else self.current_path[0]
        
        # Determine move action
        row_diff = next_pos[0] - current_pos[0]
        col_diff = next_pos[1] - current_pos[1]
        
        move_action = 'S'  # Default: stay
        if row_diff == -1:
            move_action = 'U'  # Up
        elif row_diff == 1:
            move_action = 'D'  # Down
        elif col_diff == -1:
            move_action = 'L'  # Left
        elif col_diff == 1:
            move_action = 'R'  # Right
        
        # Determine package action
        package_action = '0'  # Default: do nothing
        
        if self.destination and next_pos == self.destination:
            if self.task_type == 'pickup' and robot.carrying == 0:
                package_action = '1'  # Pickup
            elif self.task_type == 'dropoff' and robot.carrying != 0:
                package_action = '2'  # Dropoff
        
        # Update path for next time
        self.current_path = self.current_path[1:] if len(self.current_path) > 1 else []
        
        # Create space-time reservations
        reservations = []
        for i, pos in enumerate(self.current_path):
            reservations.append((pos, current_time + i + 1))
        
        return (move_action, package_action), reservations

## 3. Global Coordination System

The Global Coordination System combines the Centralized Task Manager and Robot Controllers to create a complete solution.

In [4]:
class HybridMultiRobotSystem:
    """
    Hybrid Multi-Robot Coordination System
    """
    def __init__(self, env, reassign_interval=5):
        """
        Initialize the Hybrid Multi-Robot Coordination System
        
        Parameters:
        -----------
        env : Environment
            Environment instance
        reassign_interval : int
            How often to reassign tasks (in timesteps)
        """
        self.env = env
        self.n_robots = len(env.robots)
        
        # Initialize Centralized Task Manager
        self.ctm = CentralizedTaskManager(env, reassign_interval)
        
        # Initialize Robot Controllers
        self.robot_controllers = [RobotController(i, env, self.ctm) for i in range(self.n_robots)]
    
    def get_actions(self, state):
        """
        Get actions for all robots
        
        Parameters:
        -----------
        state : dict
            Environment state
            
        Returns:
        --------
        list : List of (move_action, package_action) tuples for each robot
        """
        current_time = state['time_step']
        
        # Update task assignments
        self.ctm.update_assignments(current_time)
        
        # Prioritized planning - robots with lower IDs plan first
        space_time_reservations = {}  # (position, time) -> robot_id
        all_actions = []
        
        # Update paths for each robot (in order of ID)
        for robot_id in range(self.n_robots):
            controller = self.robot_controllers[robot_id]
            controller.update_path(current_time, space_time_reservations)
        
        # Get actions for each robot
        for robot_id in range(self.n_robots):
            controller = self.robot_controllers[robot_id]
            action, reservations = controller.get_next_action(current_time)
            all_actions.append(action)
            
            # Add reservations
            for pos, time in reservations:
                space_time_reservations[(pos, time)] = robot_id
        
        return all_actions

    def run_simulation(self, max_steps=1000, render=False):
        """
        Run the simulation
        
        Parameters:
        -----------
        max_steps : int
            Maximum number of steps to run
        render : bool
            Whether to render the environment
            
        Returns:
        --------
        dict : Simulation results
        """
        state = self.env.reset()
        total_reward = 0
        
        for t in range(max_steps):
            # Get actions
            actions = self.get_actions(state)
            
            # Take a step
            next_state, reward, done, infos = self.env.step(actions)
            total_reward += reward
            
            # Render if requested
            if render:
                self.env.render()
                print(f"Time: {t}, Reward: {reward}, Total: {total_reward}")
            
            # Update state
            state = next_state
            
            if done:
                break
        
        # Calculate statistics
        delivered_packages = [p for p in self.env.packages if p.status == 'delivered']
        successful_deliveries = sum(1 for p in delivered_packages if self.env.t <= p.deadline)
        late_deliveries = len(delivered_packages) - successful_deliveries
        success_rate = successful_deliveries / max(len(self.env.packages), 1)
        late_rate = late_deliveries / max(len(self.env.packages), 1)
        
        results = {
            'total_reward': total_reward,
            'total_steps': t + 1,
            'delivered_packages': len(delivered_packages),
            'successful_deliveries': successful_deliveries,
            'late_deliveries': late_deliveries,
            'success_rate': success_rate,
            'late_rate': late_rate
        }
        
        return results

## Testing on Different Map Configurations

Let's test our Hybrid Multi-Robot Coordination System on different map configurations.

In [13]:
def test_on_configurations():
    """
    Test the Hybrid Multi-Robot Coordination System on different configurations
    
    Returns:
    --------
    dict : Dictionary of results for each configuration
    """
    # Test configurations
    configs = [
        {"map_file": "map1.txt", "n_robots": 5, "n_packages": 100, "max_time_steps": 1000, "seed": 10},
        {"map_file": "map2.txt", "n_robots": 5, "n_packages": 100, "max_time_steps": 1000, "seed": 10},
        {"map_file": "map3.txt", "n_robots": 5, "n_packages": 500, "max_time_steps": 1000, "seed": 10},
        {"map_file": "map4.txt", "n_robots": 10, "n_packages": 500, "max_time_steps": 1000, "seed": 10},
        {"map_file": "map5.txt", "n_robots": 10, "n_packages": 1000, "max_time_steps": 1000, "seed": 10}
    ]
    
    all_results = {}
    
    for i, config in enumerate(configs):
        print(f"Testing configuration {i+1}/{len(configs)}:")
        print(f"Map: {config['map_file']}, Robots: {config['n_robots']}, Packages: {config['n_packages']}")
        
        # Create environment
        env = Environment(
            map_file=config['map_file'],
            max_time_steps=config['max_time_steps'],
            n_robots=config['n_robots'],
            n_packages=config['n_packages'],
            seed=config['seed']
        )
        
        # Create and run hybrid system
        hybrid_system = HybridMultiRobotSystem(env, reassign_interval=5)
        results = hybrid_system.run_simulation(max_steps=config['max_time_steps'], render=False)
        
        print(f"Results for configuration {i+1}:")
        print(f"Total Reward: {results['total_reward']:.2f}")
        print(f"Total Steps: {results['total_steps']}")
        print(f"Delivered Packages: {results['delivered_packages']}/{config['n_packages']}")
        print(f"Successful Deliveries: {results['successful_deliveries']}")
        print(f"Late Deliveries: {results['late_deliveries']}")
        print(f"Success Rate: {results['success_rate']:.2f}")
        print(f"Late Rate: {results['late_rate']:.2f}")
        print()
        
        all_results[f"config_{i+1}"] = results
    
    return all_results

In [14]:
def compare_with_bfs():
    """
    Compare the Hybrid Multi-Robot Coordination System with the BFS algorithm
    
    Returns:
    --------
    dict : Dictionary of results for both methods
    """
    # Test on a single configuration
    config = {"map_file": "map1.txt", "n_robots": 5, "n_packages": 100, "max_time_steps": 1000, "seed": 10}
    
    print("Testing Hybrid System:")
    # Create environment for hybrid system
    env_hybrid = Environment(
        map_file=config['map_file'],
        max_time_steps=config['max_time_steps'],
        n_robots=config['n_robots'],
        n_packages=config['n_packages'],
        seed=config['seed']
    )
    
    # Create and run hybrid system
    hybrid_system = HybridMultiRobotSystem(env_hybrid, reassign_interval=5)
    hybrid_results = hybrid_system.run_simulation(max_steps=config['max_time_steps'], render=False)
    
    print("Testing BFS Baseline:")
    # Create environment for BFS baseline
    env_bfs = Environment(
        map_file=config['map_file'],
        max_time_steps=config['max_time_steps'],
        n_robots=config['n_robots'],
        n_packages=config['n_packages'],
        seed=config['seed']
    )
    
    # Run BFS baseline
    from greedyagent import GreedyAgents
    
    state = env_bfs.reset()
    agents = GreedyAgents()
    agents.init_agents(state)
    
    total_reward = 0
    for t in range(config['max_time_steps']):
        actions = agents.get_actions(state)
        next_state, reward, done, infos = env_bfs.step(actions)
        total_reward += reward
        state = next_state
        
        if done:
            break
    
    # Calculate statistics for BFS
    delivered_packages_bfs = [p for p in env_bfs.packages if p.status == 'delivered']
    successful_deliveries_bfs = sum(1 for p in delivered_packages_bfs if env_bfs.t <= p.deadline)
    late_deliveries_bfs = len(delivered_packages_bfs) - successful_deliveries_bfs
    success_rate_bfs = successful_deliveries_bfs / max(len(env_bfs.packages), 1)
    late_rate_bfs = late_deliveries_bfs / max(len(env_bfs.packages), 1)
    
    bfs_results = {
        'total_reward': total_reward,
        'total_steps': env_bfs.t,
        'delivered_packages': len(delivered_packages_bfs),
        'successful_deliveries': successful_deliveries_bfs,
        'late_deliveries': late_deliveries_bfs,
        'success_rate': success_rate_bfs,
        'late_rate': late_rate_bfs
    }
    
    # Print comparison
    print("\nComparison:")
    print(f"{'Metric':<25} {'Hybrid System':<15} {'BFS Baseline':<15}")
    print(f"{'-'*55}")
    print(f"{'Total Reward':<25} {hybrid_results['total_reward']:<15.2f} {bfs_results['total_reward']:<15.2f}")
    print(f"{'Total Steps':<25} {hybrid_results['total_steps']:<15d} {bfs_results['total_steps']:<15d}")
    print(f"{'Delivered Packages':<25} {hybrid_results['delivered_packages']:<15d} {bfs_results['delivered_packages']:<15d}")
    print(f"{'Successful Deliveries':<25} {hybrid_results['successful_deliveries']:<15d} {bfs_results['successful_deliveries']:<15d}")
    print(f"{'Late Deliveries':<25} {hybrid_results['late_deliveries']:<15d} {bfs_results['late_deliveries']:<15d}")
    print(f"{'Success Rate':<25} {hybrid_results['success_rate']:<15.2f} {bfs_results['success_rate']:<15.2f}")
    print(f"{'Late Rate':<25} {hybrid_results['late_rate']:<15.2f} {bfs_results['late_rate']:<15.2f}")
    
    return {'hybrid': hybrid_results, 'bfs': bfs_results}

In [15]:
# Run the main test
# Uncomment the line below to run the test on all configurations
all_results = test_on_configurations()

Testing configuration 1/5:
Map: map1.txt, Robots: 5, Packages: 100
Results for configuration 1:
Total Reward: 258.83
Total Steps: 1000
Delivered Packages: 28/100
Successful Deliveries: 0
Late Deliveries: 28
Success Rate: 0.00
Late Rate: 0.28

Testing configuration 2/5:
Map: map2.txt, Robots: 5, Packages: 100
Results for configuration 2:
Total Reward: 57.64
Total Steps: 1000
Delivered Packages: 6/100
Successful Deliveries: 0
Late Deliveries: 6
Success Rate: 0.00
Late Rate: 0.06

Testing configuration 3/5:
Map: map3.txt, Robots: 5, Packages: 500
Results for configuration 2:
Total Reward: 57.64
Total Steps: 1000
Delivered Packages: 6/100
Successful Deliveries: 0
Late Deliveries: 6
Success Rate: 0.00
Late Rate: 0.06

Testing configuration 3/5:
Map: map3.txt, Robots: 5, Packages: 500
Results for configuration 3:
Total Reward: 58.03
Total Steps: 1000
Delivered Packages: 6/500
Successful Deliveries: 0
Late Deliveries: 6
Success Rate: 0.00
Late Rate: 0.01

Testing configuration 4/5:
Map: map4.

In [16]:
# Compare with BFS baseline
# Uncomment the line below to run the comparison
comparison_results = compare_with_bfs()

Testing Hybrid System:
Testing BFS Baseline:
State robot:  [(2, 8, 0), (5, 2, 0), (1, 6, 0), (8, 4, 0), (4, 6, 0)]
N robots =  5
Actions =  [('L', '0'), ('L', '1'), ('D', '0'), ('L', '0'), ('L', '0')]
[2, 4, 2, 3, 6]
State robot:  [(2, 7, 0), (5, 1, 4), (2, 6, 0), (8, 3, 0), (4, 5, 0)]
N robots =  5
Actions =  [('L', '0'), ('D', '0'), ('L', '0'), ('L', '1'), ('L', '1')]
[2, 4, 2, 3, 6]
State robot:  [(2, 6, 0), (6, 1, 4), (2, 5, 0), (8, 2, 3), (4, 4, 6)]
N robots =  5
Actions =  [('L', '0'), ('R', '2'), ('L', '1'), ('U', '0'), ('D', '0')]
[2, 4, 2, 3, 6]
State robot:  [(2, 5, 0), (6, 2, 0), (2, 4, 2), (7, 2, 3), (5, 4, 6)]
N robots =  5
Actions =  [('L', '1'), ('D', '0'), ('D', '0'), ('U', '0'), ('D', '0')]
[2, 3, 2, 3, 6]
State robot:  [(2, 4, 0), (6, 2, 0), (3, 4, 2), (7, 2, 3), (6, 4, 6)]
N robots =  5
Actions =  [('S', '1'), ('D', '0'), ('D', '0'), ('U', '0'), ('R', '0')]
[2, 3, 2, 3, 6]
State robot:  [(2, 4, 0), (6, 2, 0), (4, 4, 2), (7, 2, 3), (6, 5, 6)]
N robots =  5
Actions =  

## Visualization

Let's visualize the results of our Hybrid Multi-Robot Coordination System.

In [7]:
def visualize_results(all_results):
    """
    Visualize the results of the Hybrid Multi-Robot Coordination System
    
    Parameters:
    -----------
    all_results : dict
        Dictionary of results for each configuration
    """
    # Set up the figure
    fig, axs = plt.subplots(2, 2, figsize=(15, 10))
    
    # Extract data
    configs = list(all_results.keys())
    total_rewards = [all_results[config]['total_reward'] for config in configs]
    success_rates = [all_results[config]['success_rate'] for config in configs]
    late_rates = [all_results[config]['late_rate'] for config in configs]
    total_steps = [all_results[config]['total_steps'] for config in configs]
    
    # Plot total rewards
    axs[0, 0].bar(configs, total_rewards, color='skyblue')
    axs[0, 0].set_title('Total Reward by Configuration')
    axs[0, 0].set_xlabel('Configuration')
    axs[0, 0].set_ylabel('Total Reward')
    axs[0, 0].grid(True, linestyle='--', alpha=0.7)
    
    # Plot success rates
    axs[0, 1].bar(configs, success_rates, color='green')
    axs[0, 1].set_title('Success Rate by Configuration')
    axs[0, 1].set_xlabel('Configuration')
    axs[0, 1].set_ylabel('Success Rate')
    axs[0, 1].set_ylim(0, 1)
    axs[0, 1].grid(True, linestyle='--', alpha=0.7)
    
    # Plot late rates
    axs[1, 0].bar(configs, late_rates, color='orange')
    axs[1, 0].set_title('Late Rate by Configuration')
    axs[1, 0].set_xlabel('Configuration')
    axs[1, 0].set_ylabel('Late Rate')
    axs[1, 0].set_ylim(0, 1)
    axs[1, 0].grid(True, linestyle='--', alpha=0.7)
    
    # Plot total steps
    axs[1, 1].bar(configs, total_steps, color='purple')
    axs[1, 1].set_title('Total Steps by Configuration')
    axs[1, 1].set_xlabel('Configuration')
    axs[1, 1].set_ylabel('Total Steps')
    axs[1, 1].grid(True, linestyle='--', alpha=0.7)
    
    plt.tight_layout()
    plt.savefig('hybrid_system_results.png')
    plt.show()

In [8]:
def visualize_comparison(comparison_results):
    """
    Visualize the comparison between the Hybrid System and BFS baseline
    
    Parameters:
    -----------
    comparison_results : dict
        Dictionary of results for both methods
    """
    # Set up the figure
    fig, axs = plt.subplots(2, 2, figsize=(15, 10))
    
    # Extract data
    methods = ['Hybrid System', 'BFS Baseline']
    total_rewards = [comparison_results['hybrid']['total_reward'], comparison_results['bfs']['total_reward']]
    success_rates = [comparison_results['hybrid']['success_rate'], comparison_results['bfs']['success_rate']]
    late_rates = [comparison_results['hybrid']['late_rate'], comparison_results['bfs']['late_rate']]
    total_steps = [comparison_results['hybrid']['total_steps'], comparison_results['bfs']['total_steps']]
    
    # Plot total rewards
    axs[0, 0].bar(methods, total_rewards, color=['skyblue', 'lightgreen'])
    axs[0, 0].set_title('Total Reward')
    axs[0, 0].set_ylabel('Total Reward')
    axs[0, 0].grid(True, linestyle='--', alpha=0.7)
    
    # Plot success rates
    axs[0, 1].bar(methods, success_rates, color=['skyblue', 'lightgreen'])
    axs[0, 1].set_title('Success Rate')
    axs[0, 1].set_ylabel('Success Rate')
    axs[0, 1].set_ylim(0, 1)
    axs[0, 1].grid(True, linestyle='--', alpha=0.7)
    
    # Plot late rates
    axs[1, 0].bar(methods, late_rates, color=['skyblue', 'lightgreen'])
    axs[1, 0].set_title('Late Rate')
    axs[1, 0].set_ylabel('Late Rate')
    axs[1, 0].set_ylim(0, 1)
    axs[1, 0].grid(True, linestyle='--', alpha=0.7)
    
    # Plot total steps
    axs[1, 1].bar(methods, total_steps, color=['skyblue', 'lightgreen'])
    axs[1, 1].set_title('Total Steps')
    axs[1, 1].set_ylabel('Total Steps')
    axs[1, 1].grid(True, linestyle='--', alpha=0.7)
    
    plt.tight_layout()
    plt.savefig('comparison_results.png')
    plt.show()

In [9]:
# Visualize the results
# Uncomment the lines below to visualize the results
# if 'all_results' in locals():
#     visualize_results(all_results)

# if 'comparison_results' in locals():
#     visualize_comparison(comparison_results)

## Main Execution

Here's how to use the Hybrid Multi-Robot Coordination System.

In [10]:
def main():
    """
    Main execution function
    """
    # Create environment
    env = Environment(
        map_file='map1.txt',
        max_time_steps=1000,
        n_robots=5,
        n_packages=100,
        seed=10
    )
    
    # Create and run hybrid system
    hybrid_system = HybridMultiRobotSystem(env, reassign_interval=5)
    results = hybrid_system.run_simulation(max_steps=1000, render=True)
    
    print("Results:")
    print(f"Total Reward: {results['total_reward']:.2f}")
    print(f"Total Steps: {results['total_steps']}")
    print(f"Delivered Packages: {results['delivered_packages']}")
    print(f"Successful Deliveries: {results['successful_deliveries']}")
    print(f"Late Deliveries: {results['late_deliveries']}")
    print(f"Success Rate: {results['success_rate']:.2f}")
    print(f"Late Rate: {results['late_rate']:.2f}")
    
    return results

In [17]:
# Run the main function
# Uncomment the line below to run the main function
results = main()

1	1	1	1	1	1	1	1	1	1
1	0	0	0	0	R2	0	0	0	1
1	0	0	0	0	0	0	R0	0	1
1	0	0	0	0	0	0	0	0	1
1	0	0	0	0	R4	0	0	0	1
1	R1	0	0	0	0	0	0	0	1
1	0	0	0	0	0	0	0	0	1
1	0	0	0	R3	0	0	0	0	1
1	0	0	0	0	0	0	0	0	1
1	1	1	1	1	1	1	1	1	1
Time: 0, Reward: -0.05, Total: -0.05
1	1	1	1	1	1	1	1	1	1
1	0	0	0	R2	0	0	0	0	1
1	0	0	0	0	0	R0	0	0	1
1	0	0	0	0	0	0	0	0	1
1	0	0	0	R4	0	0	0	0	1
1	0	R1	0	0	0	0	0	0	1
1	0	0	0	R3	0	0	0	0	1
1	0	0	0	0	0	0	0	0	1
1	0	0	0	0	0	0	0	0	1
1	1	1	1	1	1	1	1	1	1
Time: 1, Reward: -0.05, Total: -0.1
1	1	1	1	1	1	1	1	1	1
1	0	0	R2	0	0	0	0	0	1
1	0	0	0	0	R0	0	0	0	1
1	0	0	0	0	0	0	0	0	1
1	0	0	R4	0	0	0	0	0	1
1	0	0	0	R3	0	0	0	0	1
1	0	R1	0	0	0	0	0	0	1
1	0	0	0	0	0	0	0	0	1
1	0	0	0	0	0	0	0	0	1
1	1	1	1	1	1	1	1	1	1
Time: 2, Reward: 9.95, Total: 9.85
1	1	1	1	1	1	1	1	1	1
1	0	R2	0	0	0	0	0	0	1
1	0	0	0	R0	0	0	0	0	1
1	0	0	0	0	0	0	0	0	1
1	0	R4	0	R3	0	0	0	0	1
1	0	0	0	0	0	0	0	0	1
1	0	R1	0	0	0	0	0	0	1
1	0	0	0	0	0	0	0	0	1
1	0	0	0	0	0	0	0	0	1
1	1	1	1	1	1	1	1	1	1
Time: 3, Reward: -0.04, Total: 9.81
1	1	1	1	1	1	1	1	1	1
1	R2	0	0	0	0	0	0

## Conclusion

The Hybrid Multi-Robot Coordination System combines the strengths of centralized task allocation and decentralized execution to create an efficient solution for the package delivery problem.

### Key Advantages

1. **Optimality**: The Hungarian algorithm ensures optimal task allocation.
2. **Efficiency**: A* pathfinding with space-time reservations minimizes travel distance and avoids collisions.
3. **Scalability**: The system can handle a large number of robots and packages.
4. **Adaptability**: The system can adapt to changing environments and new package arrivals.
5. **No Training Required**: Unlike reinforcement learning approaches, this system doesn't require training.

### Limitations

1. **Computational Complexity**: The Hungarian algorithm has O(n³) complexity, which can be demanding for large numbers of robots and packages.
2. **Limited Exploration**: Unlike learning-based approaches, this system doesn't explore potentially better strategies over time.
3. **Parameter Sensitivity**: The system's performance depends on the choice of parameters like reassignment interval and weights.

### Future Improvements

1. **Predictive Planning**: Incorporate predictions of future package arrivals for better planning.
2. **Advanced Congestion Management**: Implement more sophisticated congestion avoidance techniques.
3. **Hierarchical Planning**: Use a hierarchical approach to handle very large problem instances.
4. **Hybrid Learning**: Combine this approach with reinforcement learning for adaptive weight tuning.

This hybrid approach offers a robust and effective solution to the multi-robot package delivery problem, achieving high performance without the need for extensive training.